In [26]:

import numpy as np 
import pandas as pd 



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [28]:

from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
import sklearn
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [29]:

comp_dir = Path('../input/store-sales-time-series-forecasting')

store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)

test = pd.read_csv(
    comp_dir / 'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)

oil = pd.read_csv(
    comp_dir / 'oil.csv',
    parse_dates=['date'],
    infer_datetime_format=True,
)

In [32]:

oil = oil.set_index("date").asfreq(freq = "D")

oil["dcoilwtico"] = oil["dcoilwtico"].interpolate(limit_direction="both")

oil.head()

,dcoilwtico
date,
2013-01-01,93.140000
2013-01-02,93.140000
2013-01-03,92.970000
2013-01-04,93.120000
2013-01-05,93.146667


In [33]:
store_sales = store_sales.merge(oil, on="date")
test = test.merge(oil, on="date")

In [36]:

def series_to_supervised(data, n_in=1, n_out=1, futureArr=None, targetCol=None, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
   
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in df.columns]
       
    if futureArr != None:
        for i in range(0, n_out):
            for futureCol in futureArr:
                cols.append(df.shift(-i)[futureCol])
                if i == 0:
                    names += [('%s(t)' % (futureCol))]
                else:
                    names += [('%s(t+%d)' % (futureCol, i))]
    
    for i in range(0, n_out):
        if targetCol == None:
            cols.append(df.shift(-i))
            if i == 0:
                names += [('%s(t)' % (j)) for j in df.columns]
            else:
                names += [('%s(t+%d)' % (j, i)) for j in df.columns]
        else:
            cols.append(df.shift(-i)[targetCol])
            if i == 0:
                names += [('%s(t)' % (targetCol))]
            else:
                names += [('%s(t+%d)' % (targetCol, i))]
            

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [37]:

store_nbr_types = store_sales["store_nbr"].unique()

family_types = store_sales["family"].unique()


for store_nbr_type in store_nbr_types:
    for family_type in family_types:
        train_data = store_sales[(store_sales["store_nbr"] == store_nbr_type) & (store_sales["family"] == family_type)]
        train_data = train_data.reset_index()
        train_data = train_data.drop(columns = ["index", "date", "store_nbr", "family"])
        test_data = test[(test["store_nbr"] == store_nbr_type) & (test["family"] == family_type)]
        test_data = test_data.drop(columns = ["date", "store_nbr", "family"])
        break
    break

In [38]:

total_data = pd.concat([train_data, test_data]).drop(columns=["id"])

total_data

,sales,onpromotion,dcoilwtico
0,0.0,0,93.140000
1,2.0,0,93.140000
2,3.0,0,92.970000
3,3.0,0,93.120000
4,5.0,0,93.146667
...,...,...,...
19602,NaN,0,46.816667
21384,NaN,0,46.400000
23166,NaN,0,46.460000
24948,NaN,0,45.960000


In [39]:
feature_name = total_data.columns
scaler = preprocessing.MinMaxScaler(feature_range = (0,1))
total_data = scaler.fit_transform(total_data)
total_data = pd.DataFrame(total_data, columns=feature_name)
total_data

,sales,onpromotion,dcoilwtico
0,0.000000,0.0,0.792965
1,0.105263,0.0,0.792965
2,0.157895,0.0,0.790951
3,0.157895,0.0,0.792728
4,0.263158,0.0,0.793044
...,...,...,...
1695,NaN,0.0,0.244305
1696,NaN,0.0,0.239370
1697,NaN,0.0,0.240081
1698,NaN,0.0,0.234158


In [40]:
futureArr = ["onpromotion", "dcoilwtico"]
series_to_supervised(total_data, 5, 2, futureArr=futureArr, targetCol="sales")

,sales(t-5),onpromotion(t-5),dcoilwtico(t-5),sales(t-4),onpromotion(t-4),dcoilwtico(t-4),sales(t-3),onpromotion(t-3),dcoilwtico(t-3),sales(t-2),...,dcoilwtico(t-2),sales(t-1),onpromotion(t-1),dcoilwtico(t-1),onpromotion(t),dcoilwtico(t),onpromotion(t+1),dcoilwtico(t+1),sales(t),sales(t+1)
5,0.000000,0.0,0.792965,0.105263,0.0,0.792965,0.157895,0.0,0.790951,0.157895,...,0.792728,0.263158,0.0,0.793044,0.0,0.793359,0.0,0.793675,0.105263,0.000000
6,0.105263,0.0,0.792965,0.157895,0.0,0.790951,0.157895,0.0,0.792728,0.263158,...,0.793044,0.105263,0.0,0.793359,0.0,0.793675,0.0,0.793794,0.000000,0.105263
7,0.157895,0.0,0.790951,0.157895,0.0,0.792728,0.263158,0.0,0.793044,0.105263,...,0.793359,0.000000,0.0,0.793675,0.0,0.793794,0.0,0.792254,0.105263,0.105263
8,0.157895,0.0,0.792728,0.263158,0.0,0.793044,0.105263,0.0,0.793359,0.000000,...,0.793675,0.105263,0.0,0.793794,0.0,0.792254,0.0,0.800900,0.105263,0.105263
9,0.263158,0.0,0.793044,0.105263,0.0,0.793359,0.000000,0.0,0.793675,0.105263,...,0.793794,0.105263,0.0,0.792254,0.0,0.800900,0.0,0.798413,0.105263,0.157895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.263158,0.0,0.276126,0.315789,0.0,0.275337,0.368421,0.0,0.274547,0.210526,...,0.270994,0.368421,0.0,0.277153,0.0,0.264716,0.0,0.267914,0.473684,0.052632
1679,0.315789,0.0,0.275337,0.368421,0.0,0.274547,0.210526,0.0,0.270994,0.368421,...,0.277153,0.473684,0.0,0.264716,0.0,0.267914,0.0,0.263098,0.052632,0.315789
1680,0.368421,0.0,0.274547,0.210526,0.0,0.270994,0.368421,0.0,0.277153,0.473684,...,0.264716,0.052632,0.0,0.267914,0.0,0.263098,0.0,0.258281,0.315789,0.052632
1681,0.210526,0.0,0.270994,0.368421,0.0,0.277153,0.473684,0.0,0.264716,0.052632,...,0.267914,0.315789,0.0,0.263098,0.0,0.258281,0.0,0.253464,0.052632,0.052632


In [41]:
past_days = 50
predict_days = 16
futureArr = ["onpromotion", "dcoilwtico"]
targetCol = "sales"

train = series_to_supervised(total_data, past_days, predict_days, futureArr, targetCol)

split_ratio = 0.8

split_number = np.floor(len(train.index) * split_ratio)
split_number = np.int(split_number)

values = train.values
train = values[:split_number, :]
val = values[split_number:, :]

train_x, train_y = train[:, :-predict_days], train[:, -predict_days:]
val_x, val_y = val[:, :-predict_days], val[:, -predict_days:]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
val_x = val_x.reshape((val_x.shape[0], 1, val_x.shape[1]))

prediction_data = series_to_supervised(total_data, past_days, predict_days, futureArr, targetCol, dropnan=False).values[-17:-16, :-predict_days]
prediction_data = prediction_data.reshape((prediction_data.shape[0], 1, prediction_data.shape[1]))

print(train_x.shape, train_y.shape, val_x.shape, val_y.shape, prediction_data.shape)

(1295, 1, 182) (1295, 16) (324, 1, 182) (324, 16) (1, 1, 182)


In [43]:

model = keras.models.Sequential([
        keras.layers.LSTM(units=30, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])),
        keras.layers.Dropout(0.2),
        keras.layers.LSTM(units=30, return_sequences=True),
        keras.layers.Dropout(0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(predict_days))
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mse"])

model.summary()

Model: "sequential_342"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_684 (LSTM)              (None, 1, 30)             25560     
_________________________________________________________________
dropout_684 (Dropout)        (None, 1, 30)             0         
_________________________________________________________________
lstm_685 (LSTM)              (None, 1, 30)             7320      
_________________________________________________________________
dropout_685 (Dropout)        (None, 1, 30)             0         
_________________________________________________________________
time_distributed_342 (TimeDi (None, 1, 16)             496       
Total params: 33,376
Trainable params: 33,376
Non-trainable params: 0
_________________________________________________________________


In [44]:
early_stopping =  keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model_result = model.fit(train_x, train_y, epochs=100, batch_size=32, validation_data=(val_x, val_y), verbose=2, shuffle=False, callbacks=[early_stopping])

Epoch 1/100
41/41 - 3s - loss: 0.0261 - mse: 0.0261 - val_loss: 0.0256 - val_mse: 0.0257
Epoch 2/100
41/41 - 0s - loss: 0.0363 - mse: 0.0364 - val_loss: 0.0284 - val_mse: 0.0283
Epoch 3/100
41/41 - 0s - loss: 0.0218 - mse: 0.0218 - val_loss: 0.0226 - val_mse: 0.0225
Epoch 4/100
41/41 - 0s - loss: 0.0213 - mse: 0.0213 - val_loss: 0.0221 - val_mse: 0.0221
Epoch 5/100
41/41 - 0s - loss: 0.0206 - mse: 0.0206 - val_loss: 0.0219 - val_mse: 0.0219
Epoch 6/100
41/41 - 0s - loss: 0.0204 - mse: 0.0204 - val_loss: 0.0218 - val_mse: 0.0219
Epoch 7/100
41/41 - 0s - loss: 0.0202 - mse: 0.0202 - val_loss: 0.0218 - val_mse: 0.0218
Epoch 8/100
41/41 - 0s - loss: 0.0201 - mse: 0.0201 - val_loss: 0.0218 - val_mse: 0.0219
Epoch 9/100
41/41 - 0s - loss: 0.0200 - mse: 0.0200 - val_loss: 0.0219 - val_mse: 0.0219
Epoch 10/100
41/41 - 0s - loss: 0.0199 - mse: 0.0199 - val_loss: 0.0219 - val_mse: 0.0219
Epoch 11/100
41/41 - 0s - loss: 0.0197 - mse: 0.0198 - val_loss: 0.0219 - val_mse: 0.0219
Epoch 12/100
41/41 

In [46]:
prediction = model.predict(prediction_data)

prediction = np.squeeze(prediction) / scaler.scale_[0]

test_data["sales"] = prediction

In [ ]:
store_nbr_types = store_sales["store_nbr"].unique()
family_types = store_sales["family"].unique()

for store_nbr_type in store_nbr_types:
    for family_type in family_types:
        train_data = store_sales[(store_sales["store_nbr"] == store_nbr_type) & (store_sales["family"] == family_type)]
        train_data = train_data.reset_index()
        train_data = train_data.drop(columns = ["index", "date", "store_nbr", "family"])
        
               
        test_data = test[(test["store_nbr"] == store_nbr_type) & (test["family"] == family_type)]
        test_data = test_data.drop(columns = ["date", "store_nbr", "family"])
        total_data = pd.concat([train_data, test_data]).drop(columns=["id"])
        
       
        feature_name = total_data.columns

        scaler = preprocessing.MinMaxScaler(feature_range = (0,1))

        total_data = scaler.fit_transform(total_data)

        total_data = pd.DataFrame(total_data, columns=feature_name)
        
       
        past_days = 50
        predict_days = 16
        futureArr = ["onpromotion", "dcoilwtico"]
        targetCol = "sales"
        train = series_to_supervised(total_data, past_days, predict_days, futureArr, targetCol)
        split_ratio = 0.8
        split_number = np.floor(len(train.index) * split_ratio)
        split_number = np.int(split_number)

        values = train.values
        train = values[:split_number, :]
        val = values[split_number:, :]

       
        train_x, train_y = train[:, :-predict_days], train[:, -predict_days:]
        val_x, val_y = val[:, :-predict_days], val[:, -predict_days:]
        
        train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
        val_x = val_x.reshape((val_x.shape[0], 1, val_x.shape[1]))

      
        prediction_data = series_to_supervised(total_data, past_days, predict_days, futureArr, targetCol, dropnan=False).values[-17:-16, :-predict_days]
        prediction_data = prediction_data.reshape((prediction_data.shape[0], 1, prediction_data.shape[1]))
        
     
        model = keras.models.Sequential([
            keras.layers.LSTM(units=30, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])),
            keras.layers.Dropout(0.2),
            keras.layers.LSTM(units=30, return_sequences=True),
            keras.layers.Dropout(0.2),
            keras.layers.TimeDistributed(keras.layers.Dense(predict_days))
        ])

        optimizer = keras.optimizers.Adam(learning_rate=0.001)

        model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mse"])
        
        
        early_stopping =  keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

        model_result = model.fit(train_x, train_y, epochs=100, batch_size=32, validation_data=(val_x, val_y), verbose=0, shuffle=False, callbacks=[early_stopping])
        
              
        prediction = model.predict(prediction_data)

        prediction = np.squeeze(prediction) / scaler.scale_[0]

        test_data["sales"] = prediction
        
  
        
        


In [ ]:
submit_data = submit_data.drop(columns = ["onpromotion", "dcoilwtico"])
submit_data = submit_data.sort_values(by=["id"])
submit_data = submit_data.reset_index(drop=True)
submit_data.loc[submit_data.sales < 0.001, "sales"] = 0

In [ ]:
submit_data.to_csv('submission.csv', index=False)
